# TC tracking with UZ algorithm and TempestExtremes software
By Stella Bourdin

In [7]:
# Tracking parameters
run=nicam_gl11
zoom=8

# Define files and dirs
## Connectivity file for the zoom level
CONNECT_FILE=/home/users/sbourdin/WCRP_Hackathon/hk25-teams/hk25-TropCyc/ConnectivityFiles/ConnectivityFiles_for_healpix_zoom_$zoom.txt
## Scratch with data
scr_dir=/work/scratch-nopw2/sbourdin/
data_dir=$scr_dir/$run/data_healpix/
flist=`ls $data_dir`
## Where to store nodes
nodes_dir=$scr_dir/$run/nodes
if ! [ -d $nodes_dir ]; then mkdir $nodes_dir; fi
## Where to store tracks
tracks_dir="tracks/"
if ! [ -d $tracks_dir ]; then mkdir $tracks_dir; fi

In [8]:
## Running DetectNodes over each month
echo `date`

for f in $flist
do

    echo ----- ${f::-3} -----
    IN_FILE=$data_dir/$f
    OUT_FILE=$nodes_dir/${f::-3}.dat
    
    if ! [ -f $OUT_FILE ]; then
        conda run -n hackathon DetectNodes \
        --in_data $IN_FILE \
        --out $OUT_FILE \
        --searchbymin "psl" \
        --mergedist 6.0 \
        --closedcontourcmd "psl,200.0,5.5,0;_DIFF(zg250,zg500),-6,6.5,1.0" \
        --outputcmd "psl,min,0;_VECMAG(uas,vas),max,2;orog,min,0" \
        --in_connect $CONNECT_FILE
    else 
    echo "Nodes already computed"
    fi

done

echo `date`

Tue May 20 14:46:37 UTC 2025
----- 2020-03 -----
Arguments:
  --in_data <string> ["/work/scratch-nopw2/sbourdin//nicam_gl11/data_healpix//2020-03.nc"] 
  --in_data_list <string> [""] 
  --in_connect <string> ["/home/users/sbourdin/WCRP_Hackathon/hk25-teams/hk25-TropCyc/ConnectivityFiles/ConnectivityFiles_for_healpix_zoom_8.txt"] 
  --diag_connect <bool> [false] 
  --out <string> ["/work/scratch-nopw2/sbourdin//nicam_gl11/nodes/2020-03.dat"] 
  --out_file_list <string> [""] 
  --searchbymin <string> ["psl"] (default PSL)
  --searchbymax <string> [""] 
  --searchbythreshold <string> [""] 
  --minlon <double> [0.000000] (degrees)
  --maxlon <double> [0.000000] (degrees)
  --minlat <double> [0.000000] (degrees)
  --maxlat <double> [0.000000] (degrees)
  --minabslat <double> [0.000000] (degrees)
  --mergedist <double> [6.000000] (degrees)
  --mergeequal <bool> [false] 
  --closedcontourcmd <string> ["psl,200.0,5.5,0;_DIFF(zg250,zg500),-6,6.5,1.0"] [var,delta,dist,minmaxdist;...]
  --noclose

In [9]:
## Prepare list of nodesfiles
flist=`ls $nodes_dir/*`
cat $flist > $nodes_dir/all_nodes.dat

## StitchNodes
conda run -n hackathon StitchNodes \
--in $nodes_dir/all_nodes.dat \
--in_connect $CONNECT_FILE \
--out $tracks_dir/$run.csv \
--out_file_format "csv" \
--in_fmt "lon,lat,slp,wind,orog" \
--range 8.0 \
--mintime "54h" \
--maxgap "24h" \
--threshold "wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10;orog,<=,150.0,10" 
# NB: Double-check that the output does not mention gaps in the data
# Check that the period covered is the one you expect

rm $nodes_dir/all_nodes.dat

cat: /work/scratch-nopw2/sbourdin//nicam_gl11/nodes/all_nodes.dat: input file is output file
Arguments:
  --in <string> ["/work/scratch-nopw2/sbourdin//nicam_gl11/nodes/all_nodes.dat"] 
  --in_list <string> [""] 
  --in_connect <string> ["/home/users/sbourdin/WCRP_Hackathon/hk25-teams/hk25-TropCyc/ConnectivityFiles/ConnectivityFiles_for_healpix_zoom_8.txt"] 
  --out <string> ["tracks//nicam_gl11.csv"] 
  --in_fmt <string> ["lon,lat,slp,wind,orog"] 
  --range <double> [8.000000] (degrees)
  --mintime <string> ["54h"] 
  --time_begin <string> [""] 
  --time_end <string> [""] 
  --prioritize <string> [""] 
  --min_endpoint_dist <double> [0.000000] (degrees)
  --min_path_dist <double> [0.000000] (degrees)
  --maxgap <string> ["24h"] 
  --threshold <string> ["wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10;orog,<=,150.0,10"] [col,op,value,count;...]
  --caltype <string> ["standard"] (none|standard|noleap|360_day)
  --allow_repeated_times <bool> [false] 
  --add_velocity <bool> [false] 
  --o